In [ ]:
import threading
import queue
import time
import random

class PubSubSystem:
    def __init__(self):
        self.topics = {}
        self.lock = threading.Lock()

    def create_topic(self, topic):
        with self.lock:
            if topic not in self.topics:
                self.topics[topic] = queue.Queue()
                print(f"Topic '{topic}' created.")

    def publish(self, topic, message):
        with self.lock:
            if topic in self.topics:
                self.topics[topic].put(message)
                print(f"Message '{message}' published to topic '{topic}'.")
            else:
                print(f"Topic '{topic}' does not exist.")

    def subscribe(self, topic, subscriber_id):
        def listen():
            print(f"Subscriber {subscriber_id} listening to topic '{topic}'.")
            while True:
                if topic in self.topics:
                    message = self.topics[topic].get()
                    if message is None:  # Exit signal
                        break
                    print(f"Subscriber {subscriber_id} received message: '{message}' from topic '{topic}'.")

        thread = threading.Thread(target=listen)
        thread.start()
        return thread

    def stop_subscription(self, topic, subscriber_thread):
        with self.lock:
            if topic in self.topics:
                self.topics[topic].put(None)  # Send exit signal
                subscriber_thread.join()

# Example usage
if __name__ == "__main__":
    pubsub = PubSubSystem()

    pubsub.create_topic("sports")
    pubsub.create_topic("news")

    subscriber1 = pubsub.subscribe("sports", 1)
    subscriber2 = pubsub.subscribe("news", 2)

    def publisher(topic, messages):
        for message in messages:
            pubsub.publish(topic, message)
            time.sleep(random.random())

    publisher1 = threading.Thread(target=publisher, args=("sports", ["Game 1", "Game 2", "Game 3"]))
    publisher2 = threading.Thread(target=publisher, args=("news", ["Headline 1", "Headline 2", "Headline 3"]))

    publisher1.start()
    publisher2.start()

    publisher1.join()
    publisher2.join()

    pubsub.stop_subscription("sports", subscriber1)
    pubsub.stop_subscription("news", subscriber2)
